In [26]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os
from sklearn.model_selection import KFold

In [27]:
def load_data() -> Tuple [np.ndarray, np.ndarray, 
                          np.ndarray, np.ndarray]:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train / 255.
    x_test = x_test / 255.
    y_train = to_categorical(y_train, num_classes=10)
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = load_data()

In [28]:
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape, 
                                                                                            x_test.shape, y_test.shape))

x_train shape: (50000, 32, 32, 3) | y_train shape: (50000, 10)
x_test shape : (10000, 32, 32, 3) | y_test shape : (10000, 1)


In [29]:
input_shape = x_train[0,:,:,:].shape
model_input = Input(shape=input_shape)
print(input_shape)

(32, 32, 3)


In [30]:
def conv_pool_cnn(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [31]:
conv_pool_cnn_model = conv_pool_cnn(model_input)

In [32]:
NUM_EPOCHS = 20
def compile_and_train(model: training.Model, num_epochs: int, x_train, y_train) -> Tuple [History, str]: 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True,
                                                 save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    weight_files = glob.glob(os.path.join(os.getcwd(), 'weights/*'))
    weight_file = max(weight_files, key=os.path.getctime) # most recent file
    return history, weight_file

In [33]:
# _, conv_pool_cnn_weight_file = compile_and_train(conv_pool_cnn_model, NUM_EPOCHS)

In [34]:
def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]   
 
    return error
# (1 - evaluate_error(conv_pool_cnn_model)) * 100

In [35]:
def all_cnn(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='all_cnn')
    
    return model
all_cnn_model = all_cnn(model_input)
# _, all_cnn_weight_file = compile_and_train(all_cnn_model, NUM_EPOCHS)


# print('Error for model 2:', 1 - evaluate_error(all_cnn_model))

In [36]:
def nin_cnn(model_input: Tensor) -> training.Model:
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model

nin_cnn_model = nin_cnn(model_input)


# _, nin_cnn_weight_file = compile_and_train(nin_cnn_model, NUM_EPOCHS)

# print('Error for model 3:', 1 - evaluate_error(all_cnn_model))

In [40]:
models_train = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model]
model_weight_file = []


def bagging(data, y, n_estimators):
    print(data.shape[0])
    for m in models_train:
        choices = np.random.choice(data.shape[0], data.shape[0], replace=True)
        x_train = data[choices,:,:,:]
        y_train = y[choices, :]

        # Train each classifier on the dataset
        _, weight_file = compile_and_train(m, NUM_EPOCHS, x_train, y_train)
        model_weight_file.append(weight_file)
        print('Accuracy for model:', 1 - evaluate_error(m))       

bagging(x_train, y_train, len(models_train))

50000
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 25s 629us/step - loss: 1.8020 - acc: 0.3064 - val_loss: 1.4576 - val_acc: 0.4610
Epoch 2/20
40000/40000 [==============================] - 24s 593us/step - loss: 1.3043 - acc: 0.5213 - val_loss: 1.1141 - val_acc: 0.5918
Epoch 3/20
40000/40000 [==============================] - 24s 597us/step - loss: 1.0128 - acc: 0.6382 - val_loss: 0.9345 - val_acc: 0.6658
Epoch 4/20
40000/40000 [==============================] - 24s 602us/step - loss: 0.7991 - acc: 0.7208 - val_loss: 0.7397 - val_acc: 0.7441
Epoch 5/20
40000/40000 [==============================] - 24s 604us/step - loss: 0.6393 - acc: 0.7786 - val_loss: 0.7171 - val_acc: 0.7543
Epoch 6/20
40000/40000 [==============================] - 24s 611us/step - loss: 0.5116 - acc: 0.8228 - val_loss: 0.5943 - val_acc: 0.8037
Epoch 7/20
40000/40000 [==============================] - 25s 633us/step - loss: 0.4139 - acc: 0.8579 - val_lo

40000/40000 [==============================] - 6s 159us/step - loss: 0.9391 - acc: 0.6647 - val_loss: 0.9801 - val_acc: 0.6690
Epoch 19/20
40000/40000 [==============================] - 6s 159us/step - loss: 0.9273 - acc: 0.6719 - val_loss: 0.8897 - val_acc: 0.6873
Epoch 20/20
40000/40000 [==============================] - 6s 161us/step - loss: 0.9072 - acc: 0.6790 - val_loss: 0.8256 - val_acc: 0.7090
Error for model: 0.6658


In [41]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
all_cnn_model = all_cnn(model_input)
nin_cnn_model = nin_cnn(model_input)

conv_pool_cnn_model.load_weights(model_weight_file[0])
all_cnn_model.load_weights(model_weight_file[1])
nin_cnn_model.load_weights(model_weight_file[2])

models = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model]


# The ensemble classifier
def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

ensemble_model = ensemble(models, model_input)


print('Accuracy for ensemble model:', 1 - evaluate_error(ensemble_model))

Accuracy for ensemble model: 0.7557
